In [1]:
from os import path

import gdx2py
from gamspy_base import directory

import xl2times.main as xt
from xl2times.utils import run_gams, setup_logger

In [2]:
%pip install gdx2py gamspy-base


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
setup_logger(level=0)
input_dir = "../benchmarks/xlsx/DemoS_003/"
output_dir = "../benchmarks/out/DemoS_003-all/"
times_dir = "../TIMES_model/"

model, config = xt.read_xl(
    inputs=[input_dir],
    output_dir=output_dir,
    regions="",
    include_dummy_imports=True,
    no_cache=False,
)

/scratch/htc/skrishna/xl2times/xl2times/transforms.py:1276: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'EOH' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[
Applying transformations from ~TFM_UPD in SYSSETTINGS: 100%|██████████| 2/2 [00:00<00:00, 158.94it/s]


In [4]:
# Inspect some key parameters in the solution
def inspect_solution():
    with gdx2py.GdxFile(path.join(output_dir, "scenario.gdx"), gams_dir=directory) as g:
        print(g["REG_OBJ"].values())
        print(g["PAR_COMPRDL"].values())
        print(g["EQE_COMPRD"])

In [5]:
print(model.attributes.query('attribute == "COM_IE"')["value"])
tables = xt.to_tables(config, model)
xt.write_dd_files(tables, config, output_dir)

run_gams(times_dir, output_dir)

inspect_solution()

169    0.9
Name: value, dtype: object
2025-06-04 12:15:07.480 |  SUCCESS : Excel files successfully converted to DD and written to ../benchmarks/out/DemoS_003-all/


2025-06-04 12:15:09.090 |  SUCCESS : Ran GAMS successfully on ../benchmarks/out/DemoS_003-all/:
*** Status: Normal completion--- Job scenario.run Stop 06/04/25 12:15:08 elapsed 0:00:00.983
dict_values([3195557.668833001])
dict_values([])
None


In [6]:
# Modify COM_IE and re-run the scenario
com_ie = model.attributes.query('attribute == "COM_IE"')
model.attributes.loc[com_ie.index, "value"] = 0.95
print(model.attributes.query('attribute == "COM_IE"')["value"])

tables = xt.to_tables(config, model)
xt.write_dd_files(tables, config, output_dir)

run_gams(times_dir, output_dir)

inspect_solution()

169    0.95
Name: value, dtype: object
2025-06-04 12:15:09.540 |  SUCCESS : Excel files successfully converted to DD and written to ../benchmarks/out/DemoS_003-all/


2025-06-04 12:15:11.021 |  SUCCESS : Ran GAMS successfully on ../benchmarks/out/DemoS_003-all/:
*** Status: Normal completion--- Job scenario.run Stop 06/04/25 12:15:10 elapsed 0:00:00.936
dict_values([3095381.6789671676])
dict_values([])
None
